In [1]:
import pandas as pd
import sys
import os 

# sys.path.append("../")
# sys.path.append("../../")
# sys.path.append("../../../")



In [5]:
from router.dev.shop.components.shop_product_card_list.main import load_brand_url

load_brand_url('consortium','a.p.c.')

'https://www.consortium.co.uk/brands/a-p-c.html'

In [6]:
x = 4.237



4